In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import numpy as np
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# Any results you write to the current directory are saved as output.
# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Dataset

In [3]:
#Prepare envinromment useful variables

# Get current working directory
cwd = os.path.join("/kaggle/input")

dataset_dir = os.path.join(cwd, 'ann-and-dl-vqa/dataset_vqa')

In [4]:
import json
#open Dataset from directory
with open(os.path.join(dataset_dir,'train_data.json'), 'r') as f:
    train_data_jsonload = json.load(f)
f.close()
train_data = train_data_jsonload.get("questions")

#print(train_data)

#Qui ho inserito il validation split. NB: dobbiamo settare per bene e coerentemente tutte le dimensioni di length max + padding (nella parte di tokenizer è un puttanaio, va ripulito). 
#Al momento se inserisco il validation split, la lunghezza massima si riduce a 40 (la domanda lunga 41 finisce nel validation probabilmente) e di conseguenza nel processo di padding delle stringhe
#paddo la lunghezza fino a 40, che va in conflitto con l'input della rete che si aspetta una lunghezza fissa di 41 (che è quella giusta)

train_data[0]
dataset_size=len(train_data)

train_size = round((dataset_size/100)*80)
print(train_size)

validation_size = dataset_size-train_size
print(validation_size)

validation_data = train_data[train_size:]
print(len(validation_data))

train_data2=train_data[:train_size]
print(len(train_data))


{'answer': '2',
 'image_filename': 'CLEVR_train_000000.png',
 'question': 'How many other things are there of the same shape as the tiny cyan matte object?'}

207594
51898
51898
259492


In [5]:
#Qui creo la lista di domande, per intera, sulla quale usare il tokenizer
#varie print e cazzate di testing
from tensorflow.keras.preprocessing.text import Tokenizer

questions_list =list() #lista contenente tutte le domande 
for element in train_data:
    questions_list.append(element.get("question"))
    #print(element.get("question"))
    
#print('printo la mia lista di domande')
#print(questions_list)

#Preparing data with tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(questions_list)
vocab_size = len(tokenizer.word_index) + 1

print('printo la mia domanda più lunga')
longest_question=max(questions_list, key=len)
print(longest_question)

max_ita_length = max(len(sentence) for sentence in questions_list)
print('testing length:',max_ita_length)
longest_question_tokenized = tokenizer.texts_to_sequences(longest_question)
print('printo la mia domanda più lunga tokenizzata')
print(longest_question_tokenized)

test_list = list()
test_list.append('What number of objects are either brown things that are on the right side of the tiny brown metal cylinder or large brown matte cubes that are on the right side of the large purple rubber cylinder?')
test=tokenizer.texts_to_sequences(test_list) #texts_to_sequences ti aspetta una list di stringhe, credo 
print('printo test:')
print(test)

print('printo la lunghezza di test')
print(len(test[0]))




print("vocab size:")
print(vocab_size)

print("word_counts:")
print(tokenizer.word_counts) #mi printa le parole e il numero di volte che sono apparse in totale 

print("document_count:")
print(tokenizer.document_count)

print("word_indexes:")
print(tokenizer.word_index)#printa la parola e l'indice che gli è stato associato

question_tokenized = tokenizer.texts_to_sequences(questions_list)

ita_wtoi = tokenizer.word_index
#print('Total italian words:', len(ita_wtoi))

max_question_length = max(len(sentence) for sentence in question_tokenized)
print('Max question length:', max_question_length)

longest_question=max(question_tokenized, key=len)
print('max length tokenized')
print(longest_question)
#la lunghezza massima di una domanda è di 41 parole

printo la mia domanda più lunga
What number of objects are either brown things that are on the right side of the tiny brown metal cylinder or large brown matte cubes that are on the right side of the large purple rubber cylinder?
testing length: 197
printo la mia domanda più lunga tokenizzata
[[], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [], [27], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [27], [], [

In [6]:

# ImageDataGenerator
# ------------------
#Per la parte di stringhe, usare tokenizer. Prima estriamo TUTTE le domandee su di esse runniamo 
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(text)
#e otteniamo la tokenizzazione di tutte le parole uniche nelle nostre domande.
#Una volta fatto cio, possiamo passare tranquillamente al generator usando, per ogni stringa generata dal generator il metodo sequences = tokenizer.texts_to_sequences(question)
#che returna la stessa stringa ma con degli interi che indicano le parole.
#Fonte: https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

#NB: Da verificare se non c'è da applicare prima un text processing (rimozione punteggiatura,maiuscole ecc...), che però possiamo embeddare nel tokenizer in teoria 
#-----------------
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

img_directory= os.path.join(dataset_dir, "train")
import cv2 

thisdict = {
  '0': 0,
  '1': 1,
 '10': 2,
  '2': 3,
  '3': 4,
  '4': 5,
  '5': 6,
  '6': 7,
  '7': 8,
  '8': 9,
  '9': 10,
 'no': 11,
'yes': 12
}



def get_image(question):
    image_name = question.get("image_filename")
    img = cv2.imread(os.path.join(img_directory, image_name))
    return(img)

def get_question(question):
    question_text = question.get("question")
    return(question_text)

def question_to_token(question):
    question_list = list()
    question_list.append(question)
    question_token_list = tokenizer.texts_to_sequences(question_list)
    return(question_token_list[0])

def padding(question_token):
    # Pad to max question length
    question_token_list = list()
    question_token_list.append(question_token)
    question_padded_list = pad_sequences(question_token_list, maxlen=max_question_length, padding='post')
    return(question_padded_list[0])

def preprocess_question(question):
    question_tokenized = question_to_token(question)
    question_padded = padding(question_tokenized)
    return(question_padded)


def get_output(question):
    question_answer = question.get("answer")
    x = thisdict[question_answer]
    return(x)

def preprocess_input(image):
    """--- Rescale Image
    --- Rotate Image
    --- Resize Image
    --- Flip Image
    --- PCA etc. """   
    return(image)

def image_generator(data, batch_size = 64):
    
    while True:
        # Select files (paths/indices) for the batch
        batch_questions = np.random.choice(a = data, 
                                         size = batch_size)
        batch_input = []
        batch_output = [] 
        batch_input_image=[]
        batch_input_question=[]
        
        # Read in each input, perform preprocessing and get labels
        for input_element in batch_questions:
            input_question = get_question(input_element)
            input_question_preprocessed = preprocess_question(input_question)
            input_image = get_image(input_element)
            output = get_output(input_element) 
            input_question_preprocessed
            batch_input_image += [input_image]
            batch_input_question += [input_question_preprocessed]
            batch_output += [ output ]
          # Return a tuple of (input,output) to feed the network
        batch_x_image = np.array( batch_input_image )
        batch_x_question = np.array( batch_input_question )
        batch_y = np.array( batch_output )
        yield( [batch_x_image,batch_x_question], batch_y )


In [7]:

# Import Keras 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model, Sequential

# img shape
img_h = 320
img_w = 480
# ----------------------
EMBEDDING_SIZE = 128

# Define CNN for Image Input
vision_model = Sequential()
vision_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(img_h, img_w, 3))) #Probabile errore di dimensioni qui 
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vision_model.add(MaxPooling2D((2, 2)))
vision_model.add(Flatten())



image_input = Input(shape=(img_h, img_w, 3))
encoded_image = vision_model(image_input)

# Define RNN for language input
question_input = Input(shape=[41], dtype='int32')
embedded_question = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=EMBEDDING_SIZE, input_length=41)(question_input)
hidden_layer = LSTM(128,return_sequences=True)(embedded_question)
encoded_question = LSTM(128)(hidden_layer)


# Combine CNN and RNN to create the final model
merged = tf.keras.layers.concatenate([encoded_question, encoded_image])
intermediate = Dense(256, activation='relu')(merged)
output = Dense(13, activation='softmax')(intermediate)
vqa_model = Model(inputs=[image_input, question_input], outputs=output)   

In [8]:
# Early Stopping
# --------------
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
callbacks=[]
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
    callbacks.append(es_callback)

Using TensorFlow backend.


In [9]:
metrics = ['accuracy']
vqa_model.compile(loss='sparse_categorical_crossentropy',metrics= metrics, optimizer = 'rmsprop')


In [10]:
from keras.utils.vis_utils import plot_model

vqa_model.summary()
#plot_model(vqa_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 41)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 41, 128)      9088        input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 41, 128)      131584      embedding[0][0]                  
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 320, 480, 3) 0                                            
______________________________________________________________________________________________

In [11]:
#vqa_model.fit_generator(generator=image_generator(train_data2), callbacks=callbacks,validation_data=image_generator(validation_data), validation_steps=30, epochs=80, steps_per_epoch=30)  #fix this


In [12]:
import os
import operator
from datetime import datetime
from PIL import Image
import json

def create_csv(results, results_dir='./'):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(str(key) + ',' + str(value) + '\n')

#open Dataset from directory
with open(os.path.join(dataset_dir,'test_data.json'), 'r') as f:
    test_data_jsonload = json.load(f)
    f.close()
test_data = test_data_jsonload.get("questions")

from datetime import datetime
from PIL import Image
IMG_DIM = (320, 480)
test_img_directory=os.path.join(dataset_dir, 'test')
import cv2
def get_image(question):
    image_name = question.get("image_filename")
    img = cv2.imread(os.path.join(test_img_directory, image_name))
    return(img)
def get_question(question):
    question_text = question.get("question")
    return(question_text)
def question_to_token(question):
    question_list = list()
    question_list.append(question)
    question_token_list = tokenizer.texts_to_sequences(question_list)
    return(question_token_list[0])
def padding(question_token):
    # Pad to max question length
    question_token_list = list()
    question_token_list.append(question_token)
    question_padded_list = pad_sequences(question_token_list, maxlen=max_question_length, padding='post')
    return(question_padded_list[0])
def preprocess_question(question):
    question_tokenized = question_to_token(question)
    question_padded = padding(question_tokenized)
    return(question_padded)
results={}
index=0
"""
for question in test_data:
    question_string=preprocess_question(get_question(question))
    image = get_image(question)
    x=[image,question_string]
    #print('printo x')
    #print(x)
    #x.shape
    #https://stackoverflow.com/questions/41563720/error-when-checking-model-input-expected-convolution2d-input-1-to-have-4-dimens
    image = np.expand_dims(image, axis=0)
    out_softmax = vqa_model.predict(x=(image,np.array([question_string])))
    print(out_softmax) 
    out_softmax =  np.argmax(out_softmax)
    print('printo index')
    print(out_softmax)
    #out_softmax = np.asscalar(out_softmax)
    #print(out_softmax)
    #print('printo il valore di question')
    #print(question)
    #da correggere, la variabile name non esiste. Che indice devo usare? 
    results[question.get('question_id')] = out_softmax
create_csv(results)"""

"\nfor question in test_data:\n    question_string=preprocess_question(get_question(question))\n    image = get_image(question)\n    x=[image,question_string]\n    #print('printo x')\n    #print(x)\n    #x.shape\n    #https://stackoverflow.com/questions/41563720/error-when-checking-model-input-expected-convolution2d-input-1-to-have-4-dimens\n    image = np.expand_dims(image, axis=0)\n    out_softmax = vqa_model.predict(x=(image,np.array([question_string])))\n    print(out_softmax) \n    out_softmax =  np.argmax(out_softmax)\n    print('printo index')\n    print(out_softmax)\n    #out_softmax = np.asscalar(out_softmax)\n    #print(out_softmax)\n    #print('printo il valore di question')\n    #print(question)\n    #da correggere, la variabile name non esiste. Che indice devo usare? \n    results[question.get('question_id')] = out_softmax\ncreate_csv(results)"

In [13]:
print(results)

{}
